In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import shutil

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

In [2]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [3]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(364, 41)
Index(['lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val', 'train',
       'y_low', 'device', 'logger', 'n_test', 'run_id', 'y_high', 'log_dir',
       'n_train', 'predict', 'version', 'data_dir', 'eps_mult', 'n_splits',
       'criterion', 'do_return', 'earlystop', 'grid_size', 'noise_std',
       'only_flip', 'batch_size', 'max_epochs', 'data_kwargs', 'num_workers',
       'generate_data', 'model_summary', 'args_processed', 'do_return_model',
       'transform_kwargs', 'persistent_workers', 'run_name', 'test_loss',
       'space_length'],
      dtype='object')


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,space_length
0,0.010,TrainP,False,None,3,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,divine-spaceship-418,0.000711,NaN
1,0.010,TrainP,False,None,1,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,cosmic-universe-419,0.000709,NaN
2,0.010,TrainP,False,None,2,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,ancient-resonance-417,0.000752,NaN
3,0.001,Predict-TrainedP,False,None,3,"[mnist-predict, long]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,scarlet-sky-409,2.168962,NaN
4,0.001,Predict-TrainedP,False,None,1,"[mnist-predict, long]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,likely-brook-407,1.779646,NaN


In [4]:
api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie')
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}
print(len(run_id_api))
run_id_api

265


{'4ct2x5id': {'tags': ['dev'], 'name': 'auspicious-wonton-557'},
 'yysczszo': {'tags': ['dev'], 'name': 'chromatic-bao-556'},
 'j3ek4otr': {'tags': ['sine2d-rot'], 'name': 'lucky-lamp-537'},
 'vgsewkvn': {'tags': ['sine2d-rot'], 'name': 'glistening-dog-536'},
 'bimu03x8': {'tags': ['sine2d-rot'], 'name': 'brilliant-orchid-535'},
 'yd5lft4v': {'tags': ['sine2d-rot'], 'name': 'fortuitous-paper-534'},
 'ldmaidkj': {'tags': ['sine2d-rot'], 'name': 'bright-lamp-533'},
 '8tin3v7f': {'tags': ['sine2d-rot'], 'name': 'sweet-dog-532'},
 'f5f77hxe': {'tags': ['sine2d-rot'], 'name': 'red-chrysanthemum-531'},
 'pb2wmb8e': {'tags': ['sine2d-rot'], 'name': 'chromatic-tiger-528'},
 'qdlk2exw': {'tags': ['sine2d-rot'], 'name': 'vivid-envelope-529'},
 'bly210v3': {'tags': ['sine2d-rot'], 'name': 'twinkling-moon-530'},
 'sl6y8y39': {'tags': ['sine2d-rot'], 'name': 'glittering-pig-527'},
 '38qokx0k': {'tags': ['sine2d-rot'], 'name': 'bright-envelope-526'},
 '7fj6i512': {'tags': ['sine2d-rot'], 'name': 'gl

In [5]:
tidy_dirs = {
    'P': {
        'dir'     : os.path.join(store_dir, 'P'), 
        'listdir' : lambda dir: [(dir, dir.split('.')[0]) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: os.remove(dir),
    },
    'symlie' : {
        'dir'     : os.path.join(log_dir, 'symlie'), 
        'listdir' : lambda dir: [(dir, dir) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
    'wandb' : {
        'dir'     : os.path.join(wandb_dir), 
        'listdir' : lambda dir: [(dir, dir.split('-')[-1]) for dir in os.listdir(dir) if dir.startswith('run')],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
}

for tidy_dir, t in tidy_dirs.items():
    for dir_full, dir_strip in t['listdir'](t['dir']):
        if dir_strip not in run_id_api.keys():
            print(f'Deleting {dir_full}')
            t['rmdir'](os.path.join(t['dir'], dir_full))

In [6]:
def exceptions(runs):
    runs_new = []
    for run in runs:

        if (run.state != 'finished'): continue
        if id in results_df_old['run_id'].values: continue 
        if 'dev' in run.tags: continue
        
        print(run.id)
        runs_new.append(run)
    return runs_new

runs_selected = exceptions(runs)
print(len(runs_selected))

j3ek4otr
vgsewkvn
bimu03x8
yd5lft4v
ldmaidkj
8tin3v7f
f5f77hxe
pb2wmb8e
qdlk2exw
bly210v3
sl6y8y39
38qokx0k
7fj6i512
g7dqdwai
jgnu4i6w
twnsoxc0
ooyz9crl
1t3p6xz8
0i9j0ib5
om7zd9ac
pj1gc2g7
3pqgwl9e
dddldyj7
0ngidmi3
uon5y5u4
z9tqg82m
tpihe7gj
rjea9hcp
68md93ki
gebvkpjp
hkwx30s0
b8nj30ac
w052iyqq
18tgeccg
uuzqr41q
4xwddano
p0hftuor
36sh1dey
h66b4k49
o0gom1oq
cpi0akjx
srclf80p
2lffgtko
gg7ta8wv
2lffgtko
srclf80p
xenrzooc
rzctdym6
kat6wnqz
jzepxwuy
ruc145e0
nlcwh5am
galh6uc4
2s480z91
5xwbjwqw
65y3bgp1
b9pdd20c
jfmxmil6
y96fuwtg
rapi8jyk
8gcyo34e
2jksf5by
g7kinofv
1jqa806p
bqeq8csa
m5k1pbrn
3efv2xpb
0tqt9sr7
170eq9mv
8g1w7vzu
q3ic50qj
i1dkddgf
up6r9ere
pdfkzid7
kitezbza
u6cw81pd
87gwickx
d1rljask
8qbnp9sn
2zvdscih
np2cw13y
s7qyfq0o
ewvypada
4o6s66dg
cpu6bmf2
9ecj907t
ghjre4n4
s1l80nqk
vqc1ga7d
zoeqac33
bo0u6rro
5xraaffx
22d5kkqb
0t0e1cqn
uaxhdpz7
rex8xr0p
q3m8184p
1ousktd6
9nwnzq28
eznwkfdg
92j55rgo
ety3ctmi
gcqu51uq
f74bjwl2
x33xeio3
l5cm7g1j
oynnamyg
7akg7ozg
tv1xv5f3
ls3mo83a
b3hzpwd9
w

In [9]:
config_list = []

pbar = tqdm(runs_selected)
for run in pbar:
    id = run.id
    pbar.set_description(f'Retreiving wandb {str(id)}')

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Test loss
    try:
        test_loss_history = run.history(keys=['test_loss'])['test_loss']
        if len(test_loss_history) != 1:
            print(f'Warning: {id} has test_loss {test_loss_history}')
        # test_losses = np.array(history['test_loss'].dropna())
        # print(run.id, test_losses)
        config['test_loss'] = test_loss_history.item()
    except:
        config['test_loss'] = np.nan

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb xlqnfg5w: 100%|██████████| 217/217 [00:49<00:00,  4.42it/s]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,space_length
0,0.0010,TrainP,False,None,3,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,lucky-lamp-537,0.000743,NaN
1,0.0010,TrainP,False,None,2,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,glistening-dog-536,0.000789,NaN
2,0.0010,TrainP,False,None,1,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,brilliant-orchid-535,0.000772,NaN
3,0.0100,TrainP,False,None,3,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,fortuitous-paper-534,0.001858,NaN
4,0.0100,TrainP,False,None,2,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,bright-lamp-533,0.001986,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,0.0100,TrainP,False,None,3,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,prime-aardvark-151,0.023908,7.0
213,0.0100,TrainP,False,None,2,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,young-wind-158,0.003005,7.0
214,0.0001,TrainP,False,None,2,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,eternal-pine-146,0.009467,7.0
215,0.0010,TrainP,False,None,1,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,fragrant-plasma-147,0.000438,7.0


In [10]:
results_df.to_pickle(os.path.join(store_dir, 'results_df.pkl'))

# results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
# results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))